# Quick Start - Simple CNN Model with RPS dataset

```
Step 0. Library Import
Step 1. Dataset Load
Step 2. Data Generator
Step 3. Modeling
Step 4. Model Compile
Step 5. Save Model Checkpoint
Step 6. Model Fit
Step 7. Model Evaluate & Save
Step 8. Reload Model
```

## Step 0. Library Import

In [1]:
import urllib.request
import zipfile
import numpy as np
from IPython.display import Image

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

## Step 1. Dataset Load

In [2]:
url = 'https://storage.googleapis.com/download.tensorflow.org/data/rps.zip'
urllib.request.urlretrieve(url, './data/rps.zip')

('rps.zip', <http.client.HTTPMessage at 0x7fc6b1c09fd0>)

In [3]:
zip_ref = zipfile.ZipFile('./data/rps.zip', 'r')
zip_ref.extractall('./data')
zip_ref.close()

In [4]:
data_path = './data/rps'

## Step 2. Data Generator

In [5]:
idg = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    horizontal_flip=True,
    fill_mode='reflect',
    validation_split=0.2
)

In [7]:
batch_size = 128
target_size = (150, 150)
class_mode = 'categorical'

In [8]:
train_gen = idg.flow_from_directory(
    data_path,
    batch_size = batch_size,
    target_size = target_size,
    class_mode = class_mode,
    subset = 'training'
)

valid_gen = idg.flow_from_directory(
    data_path,
    batch_size = batch_size,
    target_size = target_size,
    class_mode = class_mode,
    subset = 'validation'
)

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.


## Step 3. Modeling

In [10]:
model = Sequential([
    Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

## Step 4. Model Complile

In [12]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

## Step 5. Save Model Checkpoint

In [13]:
filepath = 'my_checkpoint.ckpt'
cp = ModelCheckpoint(
    filepath = filepath,
    save_weights_only = True,
    save_best_only = True,
    monitor = 'val_loss',
    verbose=1
)

## Step 6. Model Fit

In [14]:
epochs = 12
model.fit(
    train_gen,
    validation_data = (valid_gen),
    epochs = epochs,
    callbacks=[cp]
)

Epoch 1/12
16/16 [==============================] - ETA: 0s - loss: 1.0535 - acc: 0.4449
Epoch 00001: val_loss improved from inf to 0.79021, saving model to my_checkpoint.ckpt
16/16 [==============================] - 37s 2s/step - loss: 1.0535 - acc: 0.4449 - val_loss: 0.7902 - val_acc: 0.7956
Epoch 2/12
16/16 [==============================] - ETA: 0s - loss: 0.4099 - acc: 0.8686
Epoch 00002: val_loss improved from 0.79021 to 0.56353, saving model to my_checkpoint.ckpt
16/16 [==============================] - 36s 2s/step - loss: 0.4099 - acc: 0.8686 - val_loss: 0.5635 - val_acc: 0.6944
Epoch 3/12
16/16 [==============================] - ETA: 0s - loss: 0.1483 - acc: 0.9435
Epoch 00003: val_loss improved from 0.56353 to 0.29365, saving model to my_checkpoint.ckpt
16/16 [==============================] - 37s 2s/step - loss: 0.1483 - acc: 0.9435 - val_loss: 0.2936 - val_acc: 0.8591
Epoch 4/12
16/16 [==============================] - ETA: 0s - loss: 0.0509 - acc: 0.9861
Epoch 00004: val_l

## Step 7. Model Evaluate & Save

In [16]:
model.load_weights(filepath)

In [18]:
model.evaluate(valid_gen)

4/4 [==============================] - 3s 692ms/step - loss: 0.0757 - acc: 0.9722


[0.07570143043994904, 0.9722222089767456]

In [19]:
model.save('./model/rps_cnn_0819.h5')

## Step 8. Reload Model

In [22]:
mymodel = tf.keras.models.load_model('./model/rps_cnn_0819.h5')

In [23]:
mymodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1